In [27]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from BitcoinLib import Bitcoin, inputs, make_input
WS = 5

In [58]:
WS = 20
instance = Bitcoin(3, [(1,2), (3,4)], [150,150], [200, 250 ,300,100],1,WS).cuda()
instance

Bitcoin(
  (varient_lstm): ModuleList(
    (0): LSTM(2, 150, num_layers=2, dropout=0.3)
    (1): LSTM(150, 1, dropout=0.2)
    (2): LSTM(2, 150, num_layers=2, dropout=0.3)
    (3): LSTM(150, 1, dropout=0.2)
  )
  (lstm): ModuleList(
    (0): LSTM(3, 200, dropout=0.2)
    (1): LSTM(200, 250, dropout=0.2)
    (2): LSTM(250, 300, dropout=0.2)
    (3): LSTM(300, 100, dropout=0.2)
  )
  (linear): Linear(in_features=100, out_features=1, bias=True)
)

In [20]:
df = pd.read_csv('F:\\ut\\8\\proje\\data\\newbit.csv').dropna()
df = df.iloc[[x for x in range(0,20000,10)]]
df

,Unnamed: 0,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,3977496,1564356000,9511.01,9521.06,9506.02,9507.01,1.146187,10905.401232,9514.505874
10,3977506,1564356600,9509.44,9511.31,9509.00,9511.31,0.231092,2197.570215,9509.499937
20,3977516,1564357200,9560.31,9560.31,9538.89,9538.90,1.015812,9691.825240,9540.960895
30,3977526,1564357800,9551.95,9551.95,9537.74,9537.74,1.983866,18926.160212,9540.041385
40,3977536,1564358400,9538.73,9551.98,9538.73,9541.89,0.794817,7584.705150,9542.706846
...,...,...,...,...,...,...,...,...,...
19950,3997646,1565565000,11518.65,11522.55,11518.65,11522.55,0.592562,6825.534522,11518.690377
19960,3997657,1565565660,11492.34,11508.36,11492.34,11508.36,0.154436,1775.087986,11494.032180
19970,3997667,1565566260,11525.00,11532.11,11496.79,11504.84,29.196789,336153.864640,11513.384837
19980,3997677,1565566860,11522.45,11522.45,11519.16,11519.16,3.455912,39815.799091,11521.067360


In [28]:
test , tarin = inputs(df)
df

,Unnamed: 0,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,3977496,1564356000,9511.01,9521.06,9506.02,9507.01,1.146187,10905.401232,9514.505874
10,3977506,1564356600,9509.44,9511.31,9509.00,9511.31,0.231092,2197.570215,9509.499937
20,3977516,1564357200,9560.31,9560.31,9538.89,9538.90,1.015812,9691.825240,9540.960895
30,3977526,1564357800,9551.95,9551.95,9537.74,9537.74,1.983866,18926.160212,9540.041385
40,3977536,1564358400,9538.73,9551.98,9538.73,9541.89,0.794817,7584.705150,9542.706846
...,...,...,...,...,...,...,...,...,...
19950,3997646,1565565000,11518.65,11522.55,11518.65,11522.55,0.592562,6825.534522,11518.690377
19960,3997657,1565565660,11492.34,11508.36,11492.34,11508.36,0.154436,1775.087986,11494.032180
19970,3997667,1565566260,11525.00,11532.11,11496.79,11504.84,29.196789,336153.864640,11513.384837
19980,3997677,1565566860,11522.45,11522.45,11519.16,11519.16,3.455912,39815.799091,11521.067360


In [59]:
instance.load_state_dict(torch.load('PN_result\\LSTM_taklaye_ws20_variantSize_150150200250300100.pn'))

<All keys matched successfully>

In [66]:
def Strategy_test():
    my_money = 10000
    my_bit = 0
    preds = []
    real = []
    with torch.no_grad():
        i = -1
        for item, y in test:
            i+=1
            instance.reset()

            preds.append(instance.forward(item))
            
            if len(preds)>1:
                if preds[-1] > preds[-2] and my_money!=0:
                    my_bit = .999*my_money/df['Open'].iloc[i-97]
                    my_money = 0
                elif preds[-1] < preds[-2] and my_bit != 0:
                    my_money = .999*my_bit * df['Open'].iloc[i-97]
                    my_bit = 0
            else:
                continue
    print(my_money,'\n', my_bit * df['Open'].iloc[-1])

In [67]:
Strategy_test()

0
10385.160344101876


In [41]:
my_money

NameError: name 'my_money' is not defined